In [6]:
# %pip install mysql-connector-python

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.5 MB 10.7 MB/s eta 0:00:02
   --- ------------------------------------ 1.3/14.5 MB 16.2 MB/s eta 0:00:01
   ------ --------------------------------- 2.2/14.5 MB 17.6 MB/s eta 0:00:01
   --------- ------------------------------ 3.3/14.5 MB 19.1 MB/s eta 0:00:01
   ------------- -------------------------- 4.9/14.5 MB 22.1 MB/s eta 0:00:01
   ---------------- ----------------------- 6.0/14.5 MB 22.4 MB/s eta 0:00:01
   ------------------- -------------------- 7.1/14.5 MB 22.6 MB/s eta 0:00:01
   ----------------------- ---------------- 8.5/14.5 MB 23.5 MB/s eta 0:00:01
   ---------------------------- ----------- 10.2/14.5 MB 24.9 MB/s eta 0:00:01
   --------------------------------- ------ 12.2/14.5 MB 31.1 MB/s eta 0:00:01
   ---------------------------------------  14.1/14.5 MB 34.6 MB/s eta 0:00:01
   ---------------------------------------  14.4/14.5 MB 34.4 MB/s et

In [36]:
import mysql.connector
import random
from datetime import datetime, timedelta

# MySQL 데이터베이스에 연결
conn = mysql.connector.connect(
    host='db-n2k14-kr.vpc-pub-cdb.ntruss.com',  # MySQL 호스트 주소 (프로토콜과 포트만 포함)
    port=3306,  # MySQL 포트 번호 (기본값 3306)
    user='user',  # MySQL 사용자 이름
    password='bankcar!@#123',  # MySQL 사용자 비밀번호
    database='bankcar'  # 사용할 데이터베이스 이름
)
cursor = conn.cursor()

In [37]:
import json
import pandas as pd

# 텍스트 파일에서 JSON 데이터 읽어오기
with open("cardata.txt", 'r', encoding='utf-8') as file:
    json_data = file.read()

# JSON 데이터 파싱
data = json.loads(json_data)

# 데이터프레임으로 변환할 리스트 초기화
dataset = []

# 각 데이터 요소에 대해 필요한 정보 추출
for item in data:
    car_id = item['metadata']['product']['id']
    doro_address = item['metadata']['depots']['pickupDepot']['name'].split('(')[0].strip()  # 띄어쓰기 이후 자르기
    model = item['props']['title']
    category = item['props']['subtitle'].split(' ') 
    if len(category) > 2:
        category = category[2]  # "또는 동급의 " 이후 잘라주기
    else:
        category = category[1] 
    price = int(item['props']['price']['current'].replace("₩", "").replace(",", "").split('.')[0])  # \ 제거하고 int로 변환
    rating = item['props']['review']['score'] if 'review' in item['props'] else 0  # 리뷰가 없을 때 None 처리
    
    # 데이터프레임으로 변환할 딕셔너리에 정보 추가
    dataset.append({
        "car_id": car_id,
        "doro_address": "서울특별시 " + doro_address,
        "model": model,
        "category": category,
        "price": int(round(price / 24, -1)),
        "rating": rating
    })

# 데이터프레임 생성
df = pd.DataFrame(dataset)

In [38]:
print(df)

       car_id                  doro_address              model category  \
0   700435863                서울특별시 서울 김포 공항   Hyundai Santa Fe      SUV   
1   700158293                서울특별시 서울 김포 공항   Hyundai Santa Fe      SUV   
2   756670446                서울특별시 서울 김포 공항       Kia Sportage      SUV   
3   756670451                서울특별시 서울 김포 공항     Hyundai Tucson      SUV   
4   700158263                     서울특별시 서대문             Kia K3       대형   
5   700158273                     서울특별시 서대문     Hyundai Avante       대형   
6   756670376                서울특별시 서울 김포 공항   Hyundai Grandeur       대형   
7   756670356                서울특별시 서울 김포 공항     Hyundai Avante       대형   
8   756670366                서울특별시 서울 김포 공항             Kia K3       대형   
9   756670461                서울특별시 서울 김포 공항        Kia Morning       소형   
10  700158218                서울특별시 서울 김포 공항             Kia K3       대형   
11  700158223                서울특별시 서울 김포 공항     Hyundai Avante       대형   
12  756670391            

In [39]:
# # 데이터 생성 함수들
# def random_date():
#     start_date = datetime(2010, 1, 1)
#     end_date = datetime.now()
#     return start_date + (end_date - start_date) * random.random()

# def generate_price(category):
#     if category == 'SUV':
#         return random.randint(80000, 150000)
#     elif category == '세단':
#         return random.randint(50000, 100000)
#     elif category == '트럭':
#         return random.randint(70000, 130000)
#     elif category == '쿠페' or category == '컨버터블':
#         return random.randint(90000, 160000)
#     elif category == '해치백':
#         return random.randint(40000, 80000)
#     elif category == '밴':
#         return random.randint(60000, 120000)
#     else:
#         return random.randint(50000, 100000)

# # categories = ['캠핑', '비지니스', '데이트', '여행', '전기차']
# colors = ['빨강', '파랑', '초록', '검정', '흰색', '노랑', '회색']

In [40]:
df['user_id'] = None
df['color'] = None
df['created_date'] = None
colors = ['빨강', '파랑', '초록', '검정', '흰색']

for index in df.index:
    df.at[index, 'user_id'] = f'user_{random.randint(1, 10000)}'
    df.at[index, 'created_date'] = datetime.now().strftime('%Y-%m-%d')
    df.at[index, 'color'] = random.choice(colors)
    
print(df.head()) 

      car_id    doro_address             model category  price rating  \
0  700435863  서울특별시 서울 김포 공항  Hyundai Santa Fe      SUV   9420    7.8   
1  700158293  서울특별시 서울 김포 공항  Hyundai Santa Fe      SUV   9370    9.1   
2  756670446  서울특별시 서울 김포 공항      Kia Sportage      SUV   6720    8.4   
3  756670451  서울특별시 서울 김포 공항    Hyundai Tucson      SUV   6720    8.4   
4  700158263       서울특별시 서대문            Kia K3       대형   4300    9.5   

     user_id color created_date  
0   user_203    검정   2024-06-07  
1  user_5147    검정   2024-06-07  
2  user_8566    파랑   2024-06-07  
3  user_3625    파랑   2024-06-07  
4  user_8957    파랑   2024-06-07  


In [41]:
for index, row in df.iterrows():
    cursor.execute('''
    INSERT IGNORE INTO CAR_DATA (car_id, doro_address, model, category, price, rating, user_id, created_date, color)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', (row['car_id'], row['doro_address'], row['model'], row['category'], row['price'], row['rating'], row['user_id'], row['created_date'], row['color']))

conn.commit()
conn.close()

print("Data inserted successfully.")

Data inserted successfully.
